In [1]:
"""
This module implements training and evaluation of a multi-layer perceptron in NumPy.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np
import os
from mlp_numpy import MLP
import cifar10_utils_py2
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

import time
import sys
import pickle



In [2]:
# Default constants
LEARNING_RATE_DEFAULT = 2e-3
WEIGHT_REGULARIZER_STRENGTH_DEFAULT = 0.
WEIGHT_INITIALIZATION_SCALE_DEFAULT = 1e-4
BATCH_SIZE_DEFAULT = 200
MAX_STEPS_DEFAULT = 1500
DNN_HIDDEN_UNITS_DEFAULT = '100'

# Directory in which cifar data is saved
DATA_DIR_DEFAULT = './cifar10/cifar-10-batches-py'
LOG_DIR = './logs/cifar10/mlp_numpy/'
MODEL_FOLDER = ''

FLAGS = None

In [6]:
def train():
    """
    Performs training and evaluation of MLP model. Evaluate your model on the whole test set each 100 iterations.
    """
    ### DO NOT CHANGE SEEDS!
    # Set the random seeds for reproducibility
    np.random.seed(42)
 

    # FLAGS.dnn_hidden_units = '100'
    ## Prepare all functions
    # Get number of units in each hidden layer specified in the string such as 100,100
    if FLAGS.dnn_hidden_units:
        dnn_hidden_units = FLAGS.dnn_hidden_units.split(",")
        dnn_hidden_units = [int(dnn_hidden_unit_) for dnn_hidden_unit_ in dnn_hidden_units]
    else:
        dnn_hidden_units = []

    train_cifar = True

    if train_cifar:
        dataset = cifar10_utils_py2.get_cifar10(FLAGS.data_dir)
        n_input = 3072
        n_classes = 10
        norm_const = 1
    else:
        dataset = input_data.read_data_sets('MNIST_data', one_hot=True)
        n_input = 784
        n_classes = 10
        norm_const = 1
        
    #SAVE FLAGS TO A FILE
    MODEL_FOLDER = LOG_DIR + 'mlp_numpy_' + time.strftime("%Y%m%d-%H%M") + '/'
    
    
    if not os.path.exists(os.path.dirname(MODEL_FOLDER)):
        try:
            os.makedirs(os.path.dirname(MODEL_FOLDER))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    file = open(MODEL_FOLDER + 'flags.txt', 'w+')
    for key, value in vars(FLAGS).items():
        file.write(key + ' : ' + str(value) + '\n')
    file.close()
    
    test_results = []


#   FLAGS.weight_reg_strength = 0.001
#   FLAGS.learning_rate = 0.1
  # FLAGS.max_steps = 10000

    mlp = MLP(n_input, dnn_hidden_units, n_classes,
            weight_decay=FLAGS.weight_reg_strength,
            weight_scale=FLAGS.weight_init_scale)

    np.set_printoptions(threshold= np.nan)

    for step in range(FLAGS.max_steps):
        x, y = dataset.train.next_batch(FLAGS.batch_size)
        x = np.reshape(x, (-1, n_input))  / norm_const

        logits = mlp.inference(x)


        loss, full_loss = mlp.loss(logits, y)
        mlp.train_step(full_loss, FLAGS)

        if step % 100 == 0:
            x = dataset.test.images
            x = np.reshape(x, (-1, n_input)) / norm_const
            y = dataset.test.labels

            logits = mlp.inference(x)
            loss, full_loss = mlp.loss(logits, y)
            acc =  mlp.accuracy(logits, y)
            print('step %d: loss: %f, %f, acc: %f' % (step, loss, full_loss,acc))
            
            test_results.append((step, full_loss, acc)) 

    #Evaluate on test set after the training has finished
    x = dataset.test.images / norm_const
    x = np.reshape(x, (-1, n_input))
    y = dataset.test.labels

    logits = mlp.inference(x)
    L, _ = mlp.loss(logits, y)
    print('test:', L, mlp.accuracy(logits, y))
    
    test_results.append((FLAGS.max_steps, full_loss, acc))  
    
    #Save results to file
    with open(MODEL_FOLDER + 'results.pkl', 'wb') as fp:
        pickle.dump(test_results, fp)
        
    print('results saved on: ' + MODEL_FOLDER)



def print_flags():
  """
  Prints all entries in FLAGS variable.
  """
  for key, value in vars(FLAGS).items():
    print(key + ' : ' + str(value))

def main():
    """
    Main function
    """
    # Print all Flags to confirm parameter settings
    print_flags()

    if not os.path.exists(FLAGS.data_dir):
        os.makedirs(FLAGS.data_dir)

    # Run the training operation

    # for layer in range(5, 0, -1):
    #   print(layer)
    # return
    train()

  

parser = argparse.ArgumentParser()
parser.add_argument('--dnn_hidden_units', type = str, default = DNN_HIDDEN_UNITS_DEFAULT,
                  help='Comma separated list of number of units in each hidden layer')
parser.add_argument('--learning_rate', type = float, default = LEARNING_RATE_DEFAULT,
                  help='Learning rate')
parser.add_argument('--max_steps', type = int, default = MAX_STEPS_DEFAULT,
                  help='Number of steps to run trainer.')
parser.add_argument('--batch_size', type = int, default = BATCH_SIZE_DEFAULT,
                  help='Batch size to run trainer.')
parser.add_argument('--weight_init_scale', type = float, default = WEIGHT_INITIALIZATION_SCALE_DEFAULT,
                  help='Weight initialization scale (e.g. std of a Gaussian).')
parser.add_argument('--weight_reg_strength', type = float, default = WEIGHT_REGULARIZER_STRENGTH_DEFAULT,
                  help='Regularizer strength for weights of fully-connected layers.')
parser.add_argument('--data_dir', type = str, default = DATA_DIR_DEFAULT,
                  help='Directory for storing input data')
FLAGS, unparsed = parser.parse_known_args()


main()


weight_init_scale : 0.0001
data_dir : ./cifar10/cifar-10-batches-py
learning_rate : 0.002
batch_size : 200
weight_reg_strength : 0.0
dnn_hidden_units : 100
max_steps : 1500
step 0: loss: 2.302503, 2.302503, acc: 0.143100
step 100: loss: 1.759325, 1.759325, acc: 0.367300
step 200: loss: 1.650975, 1.650975, acc: 0.418200
step 300: loss: 1.675326, 1.675326, acc: 0.408100
step 400: loss: 1.586292, 1.586292, acc: 0.441700
step 500: loss: 1.631646, 1.631646, acc: 0.424000
step 600: loss: 1.604198, 1.604198, acc: 0.424800
step 700: loss: 1.740624, 1.740624, acc: 0.408500
step 800: loss: 1.619057, 1.619057, acc: 0.433100
step 900: loss: 1.486658, 1.486658, acc: 0.472800
step 1000: loss: 1.517048, 1.517048, acc: 0.464500
step 1100: loss: 1.509000, 1.509000, acc: 0.476000
step 1200: loss: 1.641798, 1.641798, acc: 0.438100
step 1300: loss: 1.566983, 1.566983, acc: 0.452300
step 1400: loss: 1.494416, 1.494416, acc: 0.473600
test: 1.75098499441 0.4099
results saved on: ./logs/cifar10/mlp_numpy/mlp_

In [20]:
with open (MODEL_FOLDER + 'results.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)

In [4]:
MODEL_FOLDER

''

In [22]:
itemlist

[(0, 2.3025031447540418, 0.1431),
 (100, 1.7593248583641168, 0.36730000000000002),
 (200, 1.6509751106431194, 0.41820000000000002),
 (300, 1.675326062394713, 0.40810000000000002),
 (400, 1.5862916592989746, 0.44169999999999998),
 (500, 1.6316462978692856, 0.42399999999999999),
 (600, 1.6041976861938014, 0.42480000000000001),
 (700, 1.7406241789718035, 0.40849999999999997),
 (800, 1.6190573983364365, 0.43309999999999998),
 (900, 1.4866582967328159, 0.4728),
 (1000, 1.5170482235567944, 0.46450000000000002),
 (1100, 1.5089998029740581, 0.47599999999999998),
 (1200, 1.6417978476869572, 0.43809999999999999),
 (1300, 1.5669825193082445, 0.45229999999999998),
 (1400, 1.4944159611865293, 0.47360000000000002),
 (1500, 1.716714959303024, 0.47360000000000002)]